In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import numpy as np

In [5]:
def model_builder():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=2, activation="softmax"))
    
    return model

In [ ]:
## test the model with dummy data
net = model_builder()
net.summary()
inp = tf.random.normal(shape=(100, 784))
out = net(inp)
print(out.shape)

In [34]:
lr = 1e-3
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
def compute_and_apply_grads(inputs, targets):
    ## start recording the gradients with the GradientTape
    with tf.GradientTape() as tape:
        preds = net(inputs)
        loss = tf.keras.losses.mean_squared_error(preds, targets)
    ## compute the gradients
    grads = tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(grads, net.trainable_variables))
    ## return loss and grads for monitoring
    return loss, grads

In [35]:
def compute_and_assign_grads(inputs, targets):
    ## start recording the gradients with the GradientTape
    with tf.GradientTape() as tape:
        preds = net(inputs)
        loss = tf.keras.losses.mean_squared_error(preds, targets)
    ## compute the gradients
    grads = tape.gradient(loss, net.trainable_variables)
    #optimizer.apply_gradients(zip(grads, net.trainable_variables))
    ## apply the weight update manually
    for i in range(len(net.trainable_weights)):
        net.trainable_weights[i].assign(net.trainable_weights[i] - lr * grads[i])
    ## return loss and grads for monitoring
    return loss, grads

In [36]:
## get the mnist data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [37]:
## normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

In [38]:
## flatten and make a training and testing datasets
train_images_flat = np.array(list(map(lambda x: x.flatten(), train_images)))
test_images_flat = np.array(list(map(lambda x: x.flatten(), test_images)))
#print(train_images_flat.shape)

In [39]:
num_epochs = 10
batch_size = 100
num_batches = train_images_flat.shape[0] // batch_size
num_classes = 10

In [40]:
## batch iterate and train
for e in range(num_epochs):
    epoch_loss = list()
    for batch_id in range(num_batches):
        ## get the input and label batch
        inp_batch = train_images_flat[batch_id:batch_id+batch_size]
        #print(inp_batch.shape)
        label_batch = train_labels[batch_id:batch_id+batch_size]
        ## onehot encode the labels
        onehot_batch = tf.one_hot(label_batch, num_classes)
        #print(onehot_batch.shape)
        ## compute and apply the gradients
        loss, _ = compute_and_assign_grads(inp_batch, onehot_batch)
        epoch_loss.append(loss)
    print("Epoch Error: {}".format(np.mean(epoch_loss)))

Epoch Error: 0.04023778811097145
Epoch Error: 0.02307838387787342
Epoch Error: 0.019452393054962158
Epoch Error: 0.017360657453536987
Epoch Error: 0.015906155109405518
Epoch Error: 0.014801308512687683
Epoch Error: 0.013918668031692505
Epoch Error: 0.013190492987632751
Epoch Error: 0.012576783075928688
Epoch Error: 0.012256614863872528
